# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

/Users/aguigal/.zshenv:1: permission denied: /Library/Java/JavaVirtualMachines/jdk1.8.0_271.jdk/Contents/Home
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu==2.4.1


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_hands = mp.solutions.hands


In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections


In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                         )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [94]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [45]:
len(results.pose_landmarks.landmark)

33

In [47]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
results.pose_landmarks.landmark[0]

x: 0.4473770558834076
y: -0.27084121108055115
z: -0.7756742238998413
visibility: 0.9848765134811401

In [19]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [19]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [20]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()

    if results.face_landmarks
    else np.zeros(1404)

IndentationError: unexpected indent (1384755025.py, line 3)

In [7]:
def extract_keypoints(results):
    #face = None
    pose, lh, rh = None, None, None
    if results.pose_landmarks:
        ref = results.pose_landmarks.landmark[0]
        pose = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z, res.visibility] for res in
             results.pose_landmarks.landmark]
        ).flatten() if results.pose_landmarks else np.zeros(33 * 4)
        """face = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.face_landmarks.landmark]
        ).flatten() if results.face_landmarks else np.zeros(468 * 3)"""
        lh = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.left_hand_landmarks.landmark]
        ).flatten() if results.left_hand_landmarks else np.zeros(
            21 * 3)
        rh = np.array(
            [[res.x - ref.x, res.y - ref.y, res.z - ref.z] for res in
             results.right_hand_landmarks.landmark]
        ).flatten() if results.right_hand_landmarks else np.zeros(
            21 * 3)
    else:
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                         results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
        """face = np.array([[res.x, res.y, res.z] for res in
                         results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)"""
        lh = np.array([[res.x, res.y, res.z] for res in
                       results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(
            21 * 3)
        rh = np.array([[res.x, res.y, res.z] for res in
                       results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(
            21 * 3)
    return np.concatenate([pose, lh, rh])
    #return np.concatenate([pose, face, lh, rh])

In [22]:
result_test = extract_keypoints(results)

In [23]:
result_test

array([ 0.67701447,  0.29765448, -1.79628587, ...,  0.50258791,
        1.30048299, -0.08565646])

In [14]:
np.save('0', result_test)

NameError: name 'result_test' is not defined

In [15]:
np.load('0.npy')

array([ 0.60055482,  0.82639527, -2.14327669, ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA3')

# Actions that we try to detect
actions = np.array(['hello', 'iloveyou', 'thanks'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [9]:
DATA_PATH

'MP_DATA3'

In [39]:
for action in actions:
    if np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int).size != 0:
        dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    else:
        dirmax = 0
    for sequence in range(1, no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [10]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)

                # Change referential
                ## TODO::



                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils     import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'hello': 0, 'iloveyou': 1, 'thanks': 2}

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
np.array(sequences).shape

(90, 30, 258)

In [14]:
np.array(labels).shape

(90,)

In [15]:
X = np.array(sequences)

In [16]:
X.shape

(90, 30, 258)

In [17]:
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [19]:
y_test.shape

(5, 3)

In [20]:
X_test

array([[[ 0.        ,  0.        ,  0.        , ..., -0.35712594,
         -0.110219  ,  0.70005009],
        [ 0.        ,  0.        ,  0.        , ..., -0.35119021,
         -0.11590272,  0.66342675],
        [ 0.        ,  0.        ,  0.        , ..., -0.35418195,
         -0.11698169,  0.66319593],
        ...,
        [ 0.        ,  0.        ,  0.        , ..., -0.34902951,
         -0.23175916,  0.81786371],
        [ 0.        ,  0.        ,  0.        , ..., -0.34453031,
         -0.23501894,  0.79815623],
        [ 0.        ,  0.        ,  0.        , ..., -0.31803331,
         -0.24607158,  0.80058047]],

       [[ 0.        ,  0.        ,  0.        , ..., -0.08438385,
          0.08546719,  0.66618846],
        [ 0.        ,  0.        ,  0.        , ..., -0.48588487,
         -0.07522964,  0.68806283],
        [ 0.        ,  0.        ,  0.        , ..., -0.50760169,
         -0.09636804,  0.69669685],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

# 7. Build and Train LSTM Neural Network

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [60]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 4s 387ms/step - loss: 1.0789 - categorical_accuracy: 0.4588
Epoch 2/2000
3/3 [==============================] - 0s 39ms/step - loss: 0.8906 - categorical_accuracy: 0.6588
Epoch 3/2000
3/3 [==============================] - 0s 33ms/step - loss: 0.7419 - categorical_accuracy: 0.6235
Epoch 4/2000
3/3 [==============================] - 0s 41ms/step - loss: 0.4978 - categorical_accuracy: 0.6706
Epoch 5/2000
3/3 [==============================] - 0s 37ms/step - loss: 0.3545 - categorical_accuracy: 0.7412
Epoch 6/2000
3/3 [==============================] - 0s 30ms/step - loss: 0.5268 - categorical_accuracy: 0.7412
Epoch 7/2000
3/3 [==============================] - 0s 32ms/step - loss: 0.3117 - categorical_accuracy: 0.9647
Epoch 8/2000
3/3 [==============================] - 0s 34ms/step - loss: 0.2842 - categorical_accuracy: 0.9529
Epoch 9/2000
3/3 [==============================] - 0s 33ms/step - loss: 0.2237 - categorical_accuracy: 0.9412


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            82688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 237,251
Trainable params: 237,251
Non-trainable params: 0
__________________________________________________

# 8. Make Predictions

In [26]:
res = model.predict(X_test)

In [29]:
actions[np.argmax(res[4])]

'hello'

In [30]:
actions[np.argmax(y_test[4])]

'iloveyou'

# 9. Save Weights

In [65]:
model.save('action-test_reference2.h5')

In [42]:
del model

In [25]:
model.load_weights('action-test_reference2.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:
yhat = model.predict(X_test)

In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[2, 3],
        [0, 0]],

       [[2, 0],
        [3, 0]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

0.4

# 11. Test in Real Time

In [ ]:
!pip install fastdtw

TypeError: dtw() got an unexpected keyword argument 'dist'

In [36]:
import numpy as np

from fastdtw import fastdtw

x = np.array([1,2,3,4,5]).reshape(-1, 1)
y = np.array([3,4,5,6,7]).reshape(-1, 1)
distance, _ = fastdtw(x, y, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
print(distance)

6.0


In [37]:
def fill_blank_sequence(sequence, length, max_length):
    i = max_length - length
    j = 0
    while j < i:
        sequence.append(np.concatenate([np.zeros(33 * 4), np.zeros(21 * 3), np.zeros(21 * 3)]))
        j +=1
    return sequence

In [41]:
import numpy as np
from scipy.spatial.distance import cosine
from fastdtw import fastdtw
def compare_sequence(new_sequence, previous_sequence, limit_length, length):
    evaluate = False
    #TODO: def best parameter
    threshold = 200
    x = np.array(new_sequence).reshape(-1,1)
    y = np.array(previous_sequence).reshape(-1,1)
    distance, _ = fastdtw(x, y, dist=cosine)

    if length != 0 and distance/(length*2) > threshold:
        new_sequence = fill_blank_sequence(new_sequence[-length:], length, limit_length)
        evaluate = True
    return new_sequence, evaluate, distance

In [74]:
seq, ev, dist = compare_sequence([2,3,3,5,7, 11],[1,2,3,4], 30, 2)
len(seq)

6

In [31]:
sequence = [1,2]
sequence = fill_blank_sequence(sequence, 2, 30)
print(sequence)
len(sequence)

[1, 2, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.,

30

In [39]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [57]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1296x1296 with 0 Axes>

In [47]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.95
limit_lenght = 30
lower_lenght = 15
previous_sequence = None
new_sequence = []
current_frame_index = 0

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        #dtw for symbol with different frame
        sequence = sequence[-limit_lenght:]
        evaluate = False
        """if previous_sequence and current_frame_index < limit_lenght:
            print("===============SEQUENCE================")
            new_sequence, evaluate, dist = compare_sequence(sequence, previous_sequence, limit_lenght, current_frame_index)
            print(evaluate, dist, current_frame_index)
            print("=======================================")"""
        if current_frame_index > limit_lenght:
            #two evaluation
            first = sequence
            new_initial = current_frame_index - limit_lenght
            second = sequence[-new_initial:]
            second = fill_blank_sequence(second, new_initial, limit_lenght)
            res_first = model.predict(np.expand_dims(first, axis=0))[0]
            prob_first = np.max(res_first)
            res_second = model.predict(np.expand_dims(second, axis=0))[0]
            prob_second = np.max(res_second)
            evaluate = True
            if prob_first >= prob_second:
                new_sequence = first
                res = res_first
            else:
                new_sequence = second
                res = res_second
        elif current_frame_index >= lower_lenght:
            second = sequence[-current_frame_index:]
            second = fill_blank_sequence(second, current_frame_index, limit_lenght)
            res_second = model.predict(np.expand_dims(second, axis=0))[0]
            evaluate = True
            res = res_second

        if evaluate and len(new_sequence) == limit_lenght:
            #res = model.predict(np.expand_dims(new_sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))


        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:

                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

                    #update the new previous sequence
                    current_frame_index = -1
                    previous_sequence = new_sequence
                    sequence = new_sequence

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        current_frame_index += 1

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 0
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 1
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 2
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 3
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 4
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 5
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 6
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 7
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 8
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 9
taille 0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
taille0 0 30
frame 10
taille 0
<class 'mediapipe.python.solution_base.Sol

KeyboardInterrupt: 